**环境搭建**

In [ ]:
!nvidia-smi
import torch
import os
from ultralytics.models.yolo.detect.train import DetectionTrainer
from ultralytics import YOLO
# os.chdir(path='E://bmx/YoloCAM')

**数据集准备**

关于数据集格式的说明：标签以 YOLO 格式导出，保存为 \*.txt 文件。

如果图像中没有目标对象，则不需要创建 \*.txt 文件。 \*.txt 文件的格式为每个目标对象一行，格式为 `[类别, x_center, y_center, 宽度, 高度]`。边界框的坐标是归一化的 `[x, y, w, h]` 格式（范围从 0 到 1）。

后面的代码是以像素为单位测量的，将 `x_center` 和 `宽度` 除以图像的宽度，将 `y_center` 和 `高度` 除以图像的高度。类别编号为从 0 开始的索引（即从 0 开始计数）。

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args = dict(model="../models/yolov8-gtsrb.pt", data="./data.yaml")
trainer = DetectionTrainer(overrides=args)
trainer.setup_model()
trainer.set_model_attributes()
trainer.model.to(device=device)
trainer.model.eval()
detector = YOLO(model="../models/yolov8-gtsrb.pt")
detector.to(device=device, dtype=torch.float32).eval()

In [ ]:
from utils.dataset import CustomDataset

images_dir_path = "../datasets/gtsrb_origin/0/images"
labels_dir_path = "../datasets/gtsrb_origin/0/labels"
# Initialize dataset


train_dataset = CustomDataset(images_dir_path = images_dir_path, 
                              labels_dir_path = labels_dir_path,
                              image_width=640,
                              image_height=640)

sample_data = train_dataset[0]

import matplotlib.pyplot as plt

plt.subplot(131)
plt.imshow(sample_data["mask"][0, :, :], cmap='gray')
# img.size()
plt.subplot(132)
masked_img = sample_data["image"]
plt.imshow(masked_img.permute(1, 2, 0))
plt.subplot(133)
masked_img = sample_data["image"] * sample_data["mask"]
plt.imshow(masked_img.permute(1, 2, 0))
plt.show()

**批量攻击**

In [ ]:
import os
from gtsrb.main import classes_batch_attack_gtsrb
test_classes_root = '../../gtsrb_classes_attacked_test/origin'
output_root = "../attack_result"
os.makedirs(output_root, exist_ok=True)
classes_batch_attack_gtsrb(trainer, test_classes_root, output_root, "masked_pgd", 0.005, 0.00005, 150)

**攻击结果可视化**

In [ ]:
from utils.dataset import CustomDataset
from utils.plot import visualize_attack_result

origin_dataset = CustomDataset(images_dir_path = "../datasets/gtsrb_origin/0/images", 
                              labels_dir_path = "../datasets/gtsrb_origin/0/labels",
                              image_width=640,
                              image_height=640)
attacked_dataset = CustomDataset(images_dir_path = "../datasets/gtsrb_attack/0/images",
                              labels_dir_path = "../datasets/gtsrb_attack/0/labels",
                              image_width=640,
                              image_height=640)
visualize_attack_result(detector, origin_dataset, attacked_dataset)

**攻击效果评估**

In [ ]:
from gtsrb.main import ValGTSRB
val_gtsrb = ValGTSRB(detector, attack_result_root = "../../gtsrb_no_classes_attacked_test", origin_yaml_path='./gtsrb.yaml')
val_gtsrb.show_method()

In [ ]:
val_metrics = val_gtsrb.single_param_val(method='origin')

In [ ]:
val_metrics = val_gtsrb.single_param_val(method='fgsm', epsilon=0.03)

In [ ]:
val_metrics = val_gtsrb.single_param_val(method='masked_fgsm', epsilon=0.03)

In [ ]:
val_metrics = val_gtsrb.single_param_val(method='masked_fgsm', epsilon=0.05)

In [ ]:
val_metrics = val_gtsrb.single_param_val(method='masked_pgd', epsilon=0.05, alpha=0.0005, num_iter=100)

In [ ]:
val_metrics = val_gtsrb.single_param_val(method='masked_pgd', epsilon=0.03, alpha=0.0003, num_iter=100)

In [ ]:
val_metrics = val_gtsrb.single_param_val(method='masked_pgd', epsilon=0.02, alpha=0.0002, num_iter=100)

In [ ]:
val_metrics = val_gtsrb.single_param_val(method='masked_pgd', epsilon=0.02, alpha=0.0002, num_iter=150)

In [ ]:
val_metrics = val_gtsrb.single_param_val(method='masked_pgd', epsilon=0.01, alpha=0.0001, num_iter=100)

In [ ]:
val_metrics = val_gtsrb.single_param_val(method='masked_pgd', epsilon=0.005, alpha=0.0001, num_iter=100)

In [ ]:
val_metrics = val_gtsrb.single_param_val(method='masked_pgd', epsilon=0.005, alpha=0.0001, num_iter=150)